# L2RPN Neurips 2020 Track1 SMALL environment

In [1]:
import pathlib
from pprint import pprint

#### Set the paths for data and cofiguration file

In [2]:
LIPS_PATH = pathlib.Path().resolve().parent
DATA_PATH = LIPS_PATH / "reference_data" / "l2rpn_neurips_2020_track1_small"
LOG_PATH = LIPS_PATH / "lips_logs_neurips.log"
CONFIG_PATH = LIPS_PATH / "lips" / "config" / "conf_nips_track1.ini"

Create the path for this environment

In [3]:
if not DATA_PATH.exists():
    DATA_PATH.mkdir(mode=511, parents=False)

## Benchmark1

In [4]:
from lips.benchmark import PowerGridBenchmark
benchmark1 = PowerGridBenchmark(benchmark_path=DATA_PATH,
                                benchmark_name="Benchmark1",
                                load_data_set=False,
                                config_path=CONFIG_PATH, # opt for default config path (benchmark/config.ini)
                                log_path=LOG_PATH)

2022-05-02 12:58:36.238147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-02 12:58:36.238226: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
benchmark1.generate(nb_sample_train=int(10),
                    nb_sample_val=int(10),
                    nb_sample_test=int(10),
                    nb_sample_test_ood_topo=int(10),
                   )

test_ood_topo: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:12<00:00, 13.23s/it]


### Verfiy the chronics  (for debugging purpose, to be removed in future)

In [7]:
import re
import grid2op

In [8]:
env_nips = grid2op.make("l2rpn_neurips_2020_track1_small")

In [9]:
chronics_train_selected_regex="^((?!(.*[3-4][0-9].*)).)*$"
chronics_val_selected_regex=".*3[0-5].*"
chronics_test_selected_regex=".*3[5-9].*"
chronics_test_ood_selected_regex=".*4[0-9].*"
chronics_selected_regex = re.compile(chronics_train_selected_regex)
chronics = lambda path: re.match(chronics_selected_regex, path) is not None

In [ ]:
env_nips.chronics_handler.set_filter(chronics)
env_nips.chronics_handler.real_data.reset()

# Test (to be removed in future)

In [ ]:
benchmark1._fills_actor_simulator()

In [ ]:
benchmark1.training_actor.reference_args

In [ ]:
benchmark1.training_actor.topo_actions

In [ ]:
obs = benchmark1.env.reset()

In [ ]:
action = benchmark1.training_actor.act(obs)

In [ ]:
print(action)

In [ ]:
from lips.benchmark.utils.powergrid_utils import XDepthAgent
from lips.config import ConfigManager
import numpy as np

In [ ]:
config = ConfigManager("Benchmark1", path=CONFIG_PATH)
env = benchmark1.env

In [ ]:
for i, action in enumerate(config.get_option("dataset_create_params")["reference_args"].get("topo_actions")):
    # print(i)
    action = env.action_space(action)
    impacted_sub = int(np.where(action.get_topological_impact()[1])[0])
    print(impacted_sub)